In [5]:
from nltk.stem.lancaster import LancasterStemmer
lanstemmer = LancasterStemmer()
import nltk

import tflearn
import tensorflow as tf
import random
import json, pickle

import numpy as np

with open(r"D:/cdac/NLP/json file/intents_added.json") as file:
    data = json.load(file)

try:
    with open("datacoll.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [lanstemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lanstemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = np.array(training)
    output = np.array(output)

    with open("datacoll.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=12, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.03260 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.03260 - acc: 1.0000 -- iter: 48/55
Training Step: 5000  | total loss: 0.03290 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.03290 - acc: 1.0000 -- iter: 55/55
--
INFO:tensorflow:D:\Downloads\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
model.load('model.tflearn')

INFO:tensorflow:Restoring parameters from D:\Downloads\model.tflearn


In [8]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [lanstemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)


def start_chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() in ["quit", "end", "leave"]:
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print('YaMa: '+random.choice(responses))

start_chat()

Start talking with the bot (type quit to stop)!


You:  hello


YaMa:Yes Boss


You:  name


YaMa:I'm YaMa aka YantraManava.


...

You:  thanks


YaMa:Yes Boss


You:  awesome


YaMa:Your feedback is valuable to me


You:  end
